# Linear Solver Performance: 3D Stokes manufactured Solution after Botti and Di Pietro
### Part 1, Benchmark Setup and Execution

This benchmark was proposed in a work:

 "p‑Multilevel Preconditioners for HHO Discretizations of the Stokes Equations with Static Condensation" 
 by L. Botti and D. Di Pietro (https://doi.org/10.1007/s42967-021-00142-5)
 
It is used to assess DG as well as HDG methods, the latter beeing the main focus of the paper, the former only for reference.
The exact solution to the stationary Stokes equation is
$$
u_1 =  2 \sin(\pi x)), \\
u_2 =  -\pi y \cos(\pi x), \\
u_3 =  -\pi z \cos(\pi x), \\
  p =  \sin(\pi y) \cos(\pi y) \sin(\pi z).
$$
This velocity field is divergence-free, i.e. it fulfills the continuity equation.
To fulfill also momentum balance, a right-hand-side 
$$
g_1 =                      -2 \pi^2  \sin(\pi x), \\
g_2 =    \left( -2 \pi \cos(\pi y)^2  + \pi \right) \sin(\pi z) + \pi^3  y \cos(\pi x), \\
g_3 =   \pi \left( \cos(\pi x) \pi^2  z - \sin(\pi y) \cos(\pi y) \cos(\pi z) \right)
$$
The domain is set as $ \Omega = (0,1)^3 $.
This manufactured solution is also used as a boundary condition on three sides; 
On one side, a Neumann boundary condition is enforced. It is not mentioned which side this is. 


### Note

This example can be found in the source code repository as as `LinslvPerf_BottiPietroStokes3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

In [ ]:
#r "BoSSSpad.dll"
//#r "C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfSer") ?? "LinslvPerfSer"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

In [ ]:
wmg.Sessions

In [ ]:
/*
// extract the control object to reproduce specific simulation on local workstation
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\BenchControls";
foreach(var s in wmg.Sessions) {
    string name = s.Name;
    Console.Write(name + ": ");
    try {
        var ctrl = s.GetControl();
        var txt = ctrl.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

In [ ]:
//wmg.DefaultDatabase.Grids.ForEach(s => s.Delete(true));

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell for one variable
    public static int Np(int p) {
        //return (p*p + 3*p + 2)/2; // 2D 
        return (p*p*p + 6*p*p + 11*p + 6)/6; // 3D
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

In [ ]:
int[] Resolutions_3D = new int[] { 8, 16, 24, 32, 48, 64 };

IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(0, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(0, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-Stokes3D_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            if(Math.Abs(x - (0.0)) < 1e-8)
                return "pressure_outlet";
            return "wall"; 
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

In [ ]:
grids

In [ ]:
//PlotGrid("g2304",grids[4]);

In [ ]:
wmg.DefaultDatabase

## Setup Control Object for a Solver Run

In [ ]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Boundary Conditions and Exact Solution

Note that the 3D-example given by Botti and Di Pietro is only a manufactured solution, i.e. it fulfills the continuity, but not the momentum equation;
Hence, some non-zero right-hand-side for the momentum is required.

In [ ]:
var VelocityX = new Formula("(X) => 2.0*Math.Sin(Math.PI*X[0])");

In [ ]:
var VelocityY = new Formula("(X) => -Math.PI*X[1]*Math.Cos(Math.PI*X[0])");

In [ ]:
var VelocityZ = new Formula("(X) =>  -Math.PI*X[2]*Math.Cos(Math.PI*X[0])");

In [ ]:
var Pressure = new Formula("(X) => Math.Sin(Math.PI*X[1])*Math.Cos(Math.PI*X[1])*Math.Sin(Math.PI*X[2])");

In [ ]:
var RhsX = new Formula("(X) => -(2*Math.PI.Pow2()*Math.Sin(Math.PI*X[0]) + Math.PI*Math.Cos(Math.PI*X[0])*Math.Cos(Math.PI*X[1])*Math.Sin(Math.PI*X[2]))");

In [ ]:
var RhsY = new Formula("(X) => -(-Math.PI*(Math.Cos(Math.PI*X[0])*Math.PI.Pow2()*X[1] + Math.Sin(Math.PI*X[0])*Math.Sin(Math.PI*X[1])*Math.Sin(Math.PI*X[2])))");

In [ ]:
var RhsZ = new Formula("(X) => -(-Math.PI*(Math.Cos(Math.PI*X[0])*Math.PI.Pow2()*X[2] - Math.Cos(Math.PI*X[1])*Math.Cos(Math.PI*X[2])*Math.Sin(Math.PI*X[0])))");

In [ ]:
NUnit.Framework.Assert.Less((VelocityX.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (1.618033989)).Abs(), 1.0e-9, 
     "x-Velocity expression differs from reference value");
NUnit.Framework.Assert.Less((VelocityY.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (-0.5539745491)).Abs(), 1.0e-9, 
     "y-Velocity expression differs from reference value");
NUnit.Framework.Assert.Less((VelocityZ.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (-1.292607281)).Abs(), 1.0e-9, 
     "z-Velocity expression differs from reference value");
NUnit.Framework.Assert.Less((Pressure.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (0.3847104420)).Abs(), 1.0e-9, 
     "pressure expression differs from reference value");
NUnit.Framework.Assert.Less((RhsX.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (-16.84745722)).Abs(), 1.0e-8, 
     "Rhs of x-Momentum expression differs from reference value");
NUnit.Framework.Assert.Less((RhsY.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (+7.131009654)).Abs(), 1.0e-8, 
     "Rhs of y-Momentum expression differs from reference value");
NUnit.Framework.Assert.Less((RhsZ.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (+13.63562436)).Abs(), 1.0e-8, 
     "Rhs of z-Momentum expression differs from reference value");

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
int[] PolyDegS = new int[] {2, 3, 5};

Solvers which we want to instrument:

In [ ]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.pMultigrid
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        case LinearSolverCode.pMultigrid:
        return 700000; // 0.7 Million for direct solvers and p-MG at maximum            
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    if(solver == LinearSolverCode.pMultigrid && k <= 2)
        continue; // p-multigrid cannot really work with only two p-levels
    if(solver == LinearSolverCode.exp_gmres_levelpmg && k > 3)
        continue; // two-grid is ineffective for higher polynomial degrees
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("BottiPietroStokes3D-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    C.PhysicalParameters.rho_A             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.rho_B             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.mu_A              = 1; // dimensionless
    C.PhysicalParameters.mu_B              = 1; // dimensionless
    C.PhysicalParameters.Sigma             = 0; // not relevant, since single phase
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Boundary Conditions
    // ===================
    C.AddBoundaryValue("wall", "VelocityX", VelocityX);
    C.AddBoundaryValue("wall", "VelocityY", VelocityY);
    C.AddBoundaryValue("wall", "VelocityZ", VelocityZ);
    
    // RHS of the manufactured solution
    // ================================

    C.AddInitialValue("VolumeForceX", RhsX);
    C.AddInitialValue("VolumeForceY", RhsY);
    C.AddInitialValue("VolumeForceZ", RhsZ);

   
    // Solver Stuff
    // ============
    
    //C.VelocityBlockPrecondMode         = MultigridOperator.Mode.SymPart_DiagBlockEquilib;
    
    C.LinearSolver           = solver.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.LevelSet_ConvergenceCriterion     = 1e-6;
    C.NoOfMultigridLevels = 100;
    C.TracingNamespaces = "BoSSS.Solution";

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

Total number of simulations:

In [ ]:
controls.Count

In [ ]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [ ]:
//string path = @"C:\Users\kummer\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\release\net5.0\Stokes3D";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, "control-" + ctrl.SessionName + ".obj"));
//}

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.Activate(myBatch);
    //ctrl.RunBatch();
}

In [ ]:
wmg.AllJobs

In [ ]:
//foreach(var j in wmg.AllJobs.Values) {
//    j.DeleteOldDeploymentsAndSessions();
//}

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("BottiPietroStokes3D-J"))

In [ ]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

In [ ]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess) {
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));
    //Console.WriteLine(fail.Stderr);
    //Console.WriteLine();
    //Console.WriteLine("***************************");
    //Console.WriteLine("***************************");
    //Console.WriteLine();
}

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("BottiPietroStokes3D") && Si.SuccessfulTermination == false);
FailedSessions

In [ ]:
//FailedSessions.Select(si => (si, si.GetSessionDirectory(), si.DeployPath))

#### Asserting Success:

Remark: since this is currently (19 Aug. 2022) work-in-progress, we allow for some jobs to fail; At this intermediate milestone, I want to record (by the means of tests) what is already working. Thereby, I hope I won't break the working cases while trying to fix the failing ones.

In [ ]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [ ]:
var prelim_allowedFails = new[] {
    "BottiPietroStokes3D-J13824_p5_exp_Kcycle_schwarz"
 };

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Where(job => !prelim_allowedFails.Contains(job.Name)).Count(), "Some Jobs Failed");

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Where(s => !prelim_allowedFails.Contains(s.Name)).Count(), "Some Sessions did not terminate successfully.");